# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,barra de santo antonio,-9.4047,-35.5072,30.60,77,14,3.26,BR,2024-04-08
1,1,port-aux-francais,-49.3500,70.2167,7.54,83,79,15.20,TF,2024-04-08
2,2,lompoc,34.6391,-120.4579,23.21,62,0,6.17,US,2024-04-08
3,3,bonthe,7.5264,-12.5050,27.28,77,16,3.86,SL,2024-04-08
4,4,chonchi,-42.6128,-73.8092,11.56,70,91,2.57,CL,2024-04-08


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    c="City",
    size="Humidity",
    frame_height=550
)
# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_df=city_data_df.loc[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<27)]
wind_df=temp_df.loc[temp_df["Wind Speed"]<4.5]
narrow_df=wind_df.loc[wind_df["Cloudiness"]==0]

# Drop any rows with null values
narrow_df.dropna()

# Display sample data
narrow_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,tamanrasset,22.7850,5.5228,25.95,11,0,2.06,DZ,2024-04-08
53,53,fort bragg,35.1390,-79.0060,24.00,40,0,4.12,US,2024-04-08
74,74,marsa alam,25.0757,34.8918,22.59,52,0,2.45,EG,2024-04-08
227,227,tallahassee,30.4383,-84.2807,25.48,57,0,3.09,US,2024-04-08
252,252,stephenville,32.2207,-98.2023,26.63,60,0,3.09,US,2024-04-08
276,276,aqaba,29.5267,35.0078,23.92,38,0,1.03,JO,2024-04-08
321,321,lashio,22.9333,97.7500,21.13,46,0,1.10,MM,2024-04-08
536,536,twentynine palms,34.1356,-116.0542,21.07,24,0,3.13,US,2024-04-08
555,555,tacna,-18.0056,-70.2483,24.11,53,0,3.60,PE,2024-04-08


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=narrow_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
28,tamanrasset,DZ,22.7850,5.5228,11,
53,fort bragg,US,35.1390,-79.0060,40,
74,marsa alam,EG,25.0757,34.8918,52,
227,tallahassee,US,30.4383,-84.2807,57,
252,stephenville,US,32.2207,-98.2023,60,
276,aqaba,JO,29.5267,35.0078,38,
321,lashio,MM,22.9333,97.7500,46,
536,twentynine palms,US,34.1356,-116.0542,24,
555,tacna,PE,-18.0056,-70.2483,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":20,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index,"Lat"]
    longitude=hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    params['categories']=f"accommodation.hotel"


    # Make and API request using the params dictionaty

    name_address = requests.get(base_url,params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tamanrasset - nearest hotel: فندق الأمان
fort bragg - nearest hotel: Airborne Inn Lodging
marsa alam - nearest hotel: Blue House Hotel
tallahassee - nearest hotel: The Governors Inn
stephenville - nearest hotel: No hotel found
aqaba - nearest hotel: Nairoukh Hotel
lashio - nearest hotel: Golden Kaniri Hotel
twentynine palms - nearest hotel: Holiday Inn Express Hotel & Suites Twentynine Palms
tacna - nearest hotel: Hotel La Torre


,City,Country,Lat,Lng,Humidity,Hotel Name
28,tamanrasset,DZ,22.7850,5.5228,11,فندق الأمان
53,fort bragg,US,35.1390,-79.0060,40,Airborne Inn Lodging
74,marsa alam,EG,25.0757,34.8918,52,Blue House Hotel
227,tallahassee,US,30.4383,-84.2807,57,The Governors Inn
252,stephenville,US,32.2207,-98.2023,60,No hotel found
276,aqaba,JO,29.5267,35.0078,38,Nairoukh Hotel
321,lashio,MM,22.9333,97.7500,46,Golden Kaniri Hotel
536,twentynine palms,US,34.1356,-116.0542,24,Holiday Inn Express Hotel & Suites Twentynine ...
555,tacna,PE,-18.0056,-70.2483,53,Hotel La Torre


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    c="City",
    size="Humidity",
    frame_height=550,
    hover_cols=["Lng","Lat","City","Humidity","Hotel Name","Country"]
    
    
)
# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)